In [46]:
import os
import pandas as pd
import openpyxl
from datetime import datetime

# Directory paths
sales_mix_directory = 'sales_mix'

# Dictionary to hold the data with products as keys
products_data = {}

# Assume the data from December belongs to the previous year
current_year = datetime.now().year
previous_year = current_year - 1

In [47]:
# Process each file
for file_name in os.listdir(sales_mix_directory):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(sales_mix_directory, file_name)
        wb = openpyxl.load_workbook(file_path, data_only=True)
        ws = wb.active

        # Find the header row
        header_row_idx = None
        for rowIndex, row in enumerate(ws.iter_rows(values_only=True), start=1):
            if 'Name' in row and 'Quantity Sold' in row:
                header_row_idx = rowIndex
                headers = [cell for cell in row if cell is not None]
                name_idx = headers.index('Name')
                quantity_sold_idx = headers.index('Quantity Sold')
                break

        if header_row_idx is None:
            raise ValueError(f"'Name' and 'Quantity Sold' columns not found in the file: {file_name}")

        # Parse the date from the filename and adjust the year
        date_str = ' '.join(file_name.split(' ')[2:4]).replace(' - Copy', '')
        date_obj = datetime.strptime(date_str, '%b %d')
        if date_obj.month == 1:  # January of the next year
            date_obj = date_obj.replace(year=current_year)
        else:  # December of the previous year
            date_obj = date_obj.replace(year=previous_year)

        # Extracting data for each product
        for row in ws.iter_rows(min_row=header_row_idx+1, values_only=True):
            product_name = row[name_idx]
            quantity_sold = row[quantity_sold_idx] or 0  # Use 0 if None

            # Skip products based on names to remove or containing specific patterns
            names_to_remove = [
                "Total", "Smoothie", "Classic", "Hi Protein", "Spirit", "Superfood Plus",
                "Superfood", "Refresh", "Combo", "Regular Combo", "Snack Combo",
                "Ingredients (Smoothie)", "NO BOOSTER", "No Froyo", "No Yogurt",
                "Secret/Feature", "Specialty", "Shot", "Condiments (Booster Ball)",
                "Booster Ball", "Booster Blends", "Grilled Fresh", "Lunch", "Breakfast",
                "Merchandise", "Retail", "Fresh Juice", "Condiments (Fresh Juice)",
                "Instructions", "Condiments (Inst.)", "ALLERGY: tree nuts",
                "No Raspberry", "Special Prep", "whole wheat", "Allergy: ", "Split In 2 Cups", "Don't Make", "Almighty Acai Blend", "Condiments (Retail/Merchandise)"
, "Chipotle"                ]
            
            if any(pattern.lower() in product_name.lower() for pattern in ['allergy', 'combo', '$', 'add', 'No']):
                continue
            if product_name in names_to_remove:
                continue
            
            # Add product if not in dict
            if product_name not in products_data:
                products_data[product_name] = {}
            products_data[product_name][date_obj] = quantity_sold

            # # Add the quantity sold to the dictionary
            # date_obj[date_part][product_name] = date_product_data[date_part].get(product_name, 0) + quantity_sold


/Users/jaqueduarte/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [48]:
# Sort dates and create DataFrame with product names as columns and dates as rows
# Note: this assumes all files have the same set of products. If not, you'll need to merge them differently.
sorted_dates = sorted(list(set(date for date_dict in products_data.values() for date in date_dict)))
product_sales_df = pd.DataFrame(columns=sorted_dates)


In [49]:
# Populate the DataFrame with the sales data
for product, dates in products_data.items():
    for date, quantity in dates.items():
        product_sales_df.at[product, date] = quantity

In [50]:
# Sort the DataFrame by date (the rows)
product_sales_df.sort_index(axis=1, inplace=True)


In [51]:
# Save the DataFrame to a CSV file

output_csv_path = os.path.join('sales_mix_clean_open/combined_sales_data.csv')
product_sales_df.to_csv(output_csv_path, index_label='Product Name')


In [52]:
# Convert the DataFrame to JSON, with product names as keys
json_result = product_sales_df.to_json(orient='index', date_format='iso')

# Save the JSON result to a file
json_output_path = os.path.join('sales_mix_clean_open/combined_sales_data.json')
with open(json_output_path, 'w') as json_file:
    json_file.write(json_result)

# Return the path to the JSON file
json_output_path

'sales_mix_clean_open/combined_sales_data.json'